In [1]:
%cd ..
%pwd

/Users/nebula/Documents/MS-DataScience/Fall-2024/DL(DS-677)/Project/TradeForecast


/Users/nebula/Documents/MS-DataScience/Fall-2024/DL(DS-677)/Project/TradeForecast/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/Users/nebula/Documents/MS-DataScience/Fall-2024/DL(DS-677)/Project/TradeForecast'

In [2]:
from tradeforecast.augmentation import RNNDataset
from tradeforecast.viz import plot_rnn_forecasts
from tradeforecast.forecast.base import LitBase
from torch.utils.data import DataLoader
from lightning import Trainer
from torch import Tensor
from torch import optim
import torch.nn.functional as F

def predict(model: LitBase, data_loader: DataLoader, dataset: RNNDataset):
    y: Tensor; y_pred: Tensor
    y, y_pred = model.predict(data_loader)
    y = dataset.inverse_transform(y.numpy())
    y_pred = dataset.inverse_transform(y_pred.numpy())
    plot_rnn_forecasts(y, y_pred)
    return

In [3]:
from tradeforecast.augmentation import DataEntryPoint, Indicators, FeatureEngg
from tradeforecast.scrape import Scrapper

ticker = 'NVDA'
scrapper = Scrapper(ticker)

df_dict = scrapper.fetch_historic_data(interval='1d', start='2015-01-01', end='2024-12-06')

data_entry = DataEntryPoint(df=df_dict[ticker])

indicators = Indicators(data_entry)
indicators.add_moving_average().add_moving_average(n=30).add_macd_sl().add_rsi().add_atr()

features = FeatureEngg(data_entry)
features.add_quarters().add_weeks()

lf = data_entry.data.drop_nulls().drop('High','Low')
lf.head().collect()

Datetime,Open,Close,Volume,SMA_close_9,SMA_close_30,MACD_close_12-26,MACD-SL_close_9,RSI_close_14,ATR_close_14,Quarter_sin,Quarter_cos,Week_sin,Week_cos
"datetime[μs, EST]",f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2015-02-20 00:00:00 EST,0.531866,0.536186,157752000,0.522079,0.4922,0.013429,0.010533,75.378346,0.011963,1.0,0.0,0.812487,0.582979
2015-02-23 00:00:00 EST,0.535946,0.531866,125008000,0.526826,0.49404,0.013443,0.011185,68.533594,0.011292,1.0,0.0,0.875735,0.482792
2015-02-24 00:00:00 EST,0.533914,0.537528,149356000,0.530682,0.496005,0.013725,0.011741,72.332929,0.010908,1.0,0.0,0.875735,0.482792
2015-02-25 00:00:00 EST,0.535359,0.533432,123820000,0.534456,0.498033,0.013479,0.012114,65.710435,0.01035,1.0,0.0,0.875735,0.482792
2015-02-26 00:00:00 EST,0.533914,0.534878,152864000,0.534417,0.500134,0.013241,0.012353,66.943,0.009965,1.0,0.0,0.875735,0.482792


In [4]:
from tradeforecast.augmentation import train_val_test_split

dataset_kwargs = {'lf': lf,
                 'non_temporal': [x for x in data_entry.non_temporal if x not in ['High', 'Low']],
                 'temporal': data_entry.temporal,
                 'target': 'Close',
                 'look_back_len': 60,
                 'forecast_len': 5}

rnn_dataset = RNNDataset(**dataset_kwargs)

train_dataset, test_dataset = train_val_test_split(rnn_dataset, test_size=0.1)

batch_size = 128
num_workers = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=num_workers, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=num_workers, persistent_workers=True)

## Train LSTM model

In [ ]:
from tradeforecast.forecast import LSTM
from tradeforecast.forecast.utils import OverrideEpochStepCallback

lstm_kwargs = {'input_size': len(rnn_dataset.features),
              'hidden_size': 4,
              'n_LSTM': 2,
              'bidirectional': False,
              'fc_out_size':[],
              'output_size': rnn_dataset.forecast_len,
              'dropout': 0,
              'criterion': F.mse_loss,
              'lr': 0.1,
              'optimizer': optim.SGD}

lstm_model = LSTM(**lstm_kwargs)

lstm_trainer = Trainer(fast_dev_run=True, max_epochs=4, log_every_n_steps=10, check_val_every_n_epoch=100)

lstm_trainer.fit(lstm_model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
/Users/nebula/Documents/MS-DataScience/Fall-2024/DL(DS-677)/Project/TradeForecast/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name      | Type       | Params | Mode 
-------------------------------------------------
0 | lstm      | LSTM       | 464    | train
1 | fc_linear | Sequential | 25     | train
-------------------------------------------------
489       Trainable params
0         Non-trainable params
489       Total params
0.002     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 1/1 [00:02<00:00,  0.43it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:02<00:00,  0.43it/s]


In [6]:
y_actual, y_pred = lstm_model.predict(test_loader)

In [7]:
from tradeforecast.forecast import calc_metrics

lstm_metrics = calc_metrics(y_actual, y_pred)
lstm_metrics

{'MAE': np.float32(5.5261393),
 'MSE': np.float32(32.84177),
 'RMSE': np.float32(5.730774),
 'R-squared': -13.628268241882324}